# Generate in-silico data from a methylation reference matrix

In this folder, there is one script named gen_bedmethyl.py that allows you to generate artificial data from a methylation reference matrix. It is not directly usable from the command line, but you can use from this notebook or modify the script parameters and run it.

In [1]:
import os
import numpy as np
import pandas as pd
import numpy.random as rd

## In-silico mixture functions

The following function generates in-silico mixtures:
- gen_param_u, generates data for a given unknown portion in potentially two ways
    - Semi in-silico where one generates mixtures from a methylation pre-existing reference matrix
    - Full in-silico where one generates mixtures from a methylation reference matrix that is also randomly generated

In [2]:
import numpy as np
import numpy.random as rd

def gen_param_u(mode, nb_cpg=None, R_full=None, read_depth=100, trunc=2, unknown=0.1, nb_samples=10,
                read_depth_ref=40, nb_real_u=2):
    if mode == 'semi-silico':
        if R_full is None:
            raise ValueError("R_full must be provided for semi-silico mode.")
        nb_cpg, nb_celltypes = R_full.shape

    elif mode == 'full-in-silico':
        if nb_cpg is None:
            raise ValueError("nb_cpg must be provided for full in silico mode.")
        
        temp = np.ones((nb_cpg, trunc + nb_real_u))
        rand_param = rd.uniform(size=(1, trunc + nb_real_u))
        R_full = rd.beta(temp * rand_param, temp * rand_param)
        R_full_d = rd.poisson(read_depth_ref, (nb_cpg, trunc + nb_real_u)) + 1
        R_full_m = rd.binomial(R_full_d, R_full)
        R_full_p = R_full_m / R_full_d
        R_full = R_full_p + ((R_full_p == 0) * 1e-10) - ((R_full_p == 1) * 1e-10)
        nb_celltypes = R_full.shape[1]
    else:
        raise ValueError("Mode must be 'semi-silico' or 'full-in-silico'.")

    alpha_sim_1 = rd.dirichlet(np.ones(trunc), nb_samples).T
    alpha_sim_2 = rd.dirichlet(np.ones(nb_celltypes - trunc), 1).T
    alpha_sim = np.concatenate([alpha_sim_1 * (1 - unknown), alpha_sim_2 * unknown])

    d_x = rd.poisson(read_depth, (nb_cpg, nb_samples)) + 1
    beta_sim = R_full @ alpha_sim
    x = rd.binomial(d_x, beta_sim)
    m_u = R_full[:, trunc:] @ alpha_sim_2
    
    if mode == 'semi-silico':
        return x, d_x, np.concatenate([alpha_sim[:trunc, :], unknown]), m_u, R_full[:, :trunc]
    elif mode == 'full-in-silico':
        return x, d_x, np.concatenate([alpha_sim[:trunc, :], unknown]), m_u, R_full[:, :trunc], R_full_d[:, :trunc], R_full_m[:, :trunc], R_full
        

## Parameters and data generation

### Parameters for Data Generation

- **ref_file**:  
  Path to the methylation reference matrix to use for data generation. This file is used in the semi-silico mode to provide a predefined reference matrix.

- **random_subsample**:  
  Number of CpG sites to randomly select from the reference matrix for downsampling, if needed. This allows reducing the size of the reference matrix for computational efficiency or specific experimental needs.

- **read_depth**:  
  Average read depth for methylation reads. This determines the sequencing coverage simulated for each CpG site.

- **nb_samples**:  
  Number of samples to generate for the simulation.

- **gen_u**:  
  Specifies how to handle unknown cell types in the data generation. Possible values:
  - `None`: Do not include unknown cell types in the generated data.
  - `"random"`: Randomly select `nb_known_cell_types` columns from the reference matrix as known cell types, and the rest are treated as unknown.
  - `"first"`: Use the first `nb_known_cell_types` columns of the reference matrix as known cell types.
  - `"select"`: Use the cell types listed in the `select_cell_types` parameter as the known cell types, treating the remaining columns as unknown.

- **nb_known_cell_types**:  
  Number of known cell types to include in the simulation. This is used when selecting or generating the reference matrix.

- **select_cell_types**:  
  A list of specific cell type names to include as known cell types when `gen_u="select"` is chosen.

- **outdir**:  
  Path to the output directory where the generated files (e.g., proportions, methylation profiles, samples) will be saved. If the directory does not exist, it will be created automatically.

- **nb_cpg** *(optional)*:  
  Required only in full in silico mode. Specifies the number of CpG sites to generate the reference matrix if no pre-existing matrix (`ref_file`) is used.

- **unknown_portion**:  
  A matrix or array specifying the proportion of unknown cell types for each sample. This determines the mixing ratio of unknown and known cell types in the simulation.


In [3]:
ref_file = "bed1_select_ref_intersect.bed"
random_subsample = 350
read_depth = 50
nb_samples = 10
gen_u = "select"
unknown_portion = np.reshape(np.array([0.4, 0.2, 0.1, 0.8, 0.5, 0.1, 0.0, 0.7, 0.5, 0.9]), (1,nb_samples))
nb_known_cell_types = 5
select_cell_types = ['Adipocytes', 'Cortical_neurons', 'Hepatocytes', 'Lung_cells', 'Pancreatic_beta_cells'] 
outdir = "output_gen"

### Data generation and saving them

In [4]:
output_folder = os.path.join(os.getcwd(), outdir)
if not os.path.exists(output_folder):
    print(f'Creating directory {output_folder} to store results')
    os.mkdir(output_folder)

ref = pd.read_csv(ref_file, sep='\t')
ref = ref.sample(n=random_subsample)
pos, df = ref.iloc[:,:3], ref.iloc[:,3:]

if gen_u:
    if gen_u == "random":
        known_cell_types = random_column_names = list(rd.choice(df.columns, nb_known_cell_types, replace=False))
    elif gen_u == "first":
        known_cell_types = list(df.columns)[:nb_known_cell_types]
    elif gen_u == "select":
        known_cell_types = select_cell_types
        
    df = df[known_cell_types + [col for col in df.columns if col not in known_cell_types]]
    
    meth_counts, counts, alpha_sim, meth_u, R_known = gen_param_u(
        mode='semi-silico',
        R_full=df.values,
        read_depth=read_depth,
        trunc=nb_known_cell_types,
        unknown=unknown_portion,
        nb_samples=nb_samples
    )   

else:
    known_cell_types = df.columns
    meth_counts, counts, alpha_sim = gen_param(df.values, read_depth, nb_samples)

ref_ = pos.copy()
ref_[known_cell_types] = df[known_cell_types]
ref_.to_csv(output_folder + '/ref_matrix.bed',  sep='\t', index = False)
alpha_sim_df = pd.DataFrame(alpha_sim)
index_name = known_cell_types
if gen_u:
    index_name += ["unknown_cell_1"]
    meth_u_df = pd.DataFrame(meth_u)
    meth_u_df.columns = ["unknown_cell_1"]
    meth_u_df.to_csv(output_folder + '/meth_profile_sim.csv',  sep='\t', index = False)
alpha_sim_df.index = index_name
alpha_sim_df.columns = ["sample" + str(i + 1) for i in range(nb_samples)]
alpha_sim_df.to_csv(output_folder + '/proportions_sim.csv',  sep='\t', index = True)

for i in range(nb_samples):
    sample = pos.copy()
    sample['valid_coverage'] = counts[:,i:i+1]
    sample['count_modified'] = meth_counts[:,i:i+1]
    sample['percent_modified'] =  (sample['count_modified'] / sample['valid_coverage']) * 100

    sample.to_csv(output_folder + '/sample' + str(i + 1) + '.bed',  sep='\t', index = False)

Creating directory /Users/mbourdim/Desktop/tfk/DeMethify/test/output_gen to store results
